In [19]:
import os
import shutil
import GEOparse
import numpy as np
import pandas as pd
from ftplib import FTP
from datetime import datetime

In [20]:
def list_to_string(row):
    new_row = []
    for item in row:
        if type(item) is list:
            if len(item) > 1:
                new_item = ', '.join(item)
                new_row.append(new_item)
            else:
                new_row.append(item[0])
        else:
            new_row.append(item)
    return new_row

In [21]:
data_dir = '/Users/robertmoseley/Desktop/Haase_lab/mass_norm/data'

In [26]:
data_table = pd.read_csv('../haase_lab_microarray_datasets.csv', index_col=0)
geo_datasets = data_table.loc[: , ['GEO Accession', 'GEO name']]
geo_datasets = geo_datasets[geo_datasets['GEO Accession'] != 'No GEO']
geo_datasets

GEO Accession  \
Dropbox name                               
cdc28_r1                        GSE32974   
cdc28_r2                        GSE32974   
wt_37deg                        GSE32974   
cdc20_r1                        GSE49650   
cdc20_r2                        GSE49650   
cdc8_r1                         GSE49650   
cdc8_r2                         GSE49650   
cdc8cdc20                       GSE49650   
gal_cse4_353_r1                 GSE49650   
gal_cse4_353_r2                 GSE49650   
rad53 _r1                       GSE49650   
rad53_r2                        GSE49650   
cdc16cdc28whi5stb1_rep1         GSE75694   
cdc16cdc28whi5stb1_rep2         GSE75694   
cdc28_whi5_stb1_r1              GSE75694   
cdc28_whi5_stb1_r1              GSE75694   
cdc28_whi5_stb1_r2              GSE75694   
cdc28_whi5_stb1_r2              GSE75694   
gal_sic1_3P_r1                  GSE75694   
gal_sic1_3P_r2                  GSE75694   
gal_sic1_3P_yox1_yhp1_nrm1      GSE75694   
mt_r1                            GSE8799   
mt_r2                            GSE8799   
wt_r1                            GSE8799   
wt_r2                            GSE8799   
cdc14                           GSE96997   
cdc15                           GSE96997   

                                                          GEO name  
Dropbox name                                                        
cdc28_r1                                       cdc28-4, 37 degrees  
cdc28_r2                                       cdc28-4, 37 degrees  
wt_37deg                                     wild-type, 37 degrees  
cdc20_r1                                          Cerevisiae_cdc20  
cdc20_r2                                          Cerevisiae_cdc20  
cdc8_r1                                            Cerevisiae_cdc8  
cdc8_r2                                            Cerevisiae_cdc8  
cdc8cdc20                                     Cerevisiae_cdc8cdc20  
gal_cse4_353_r1                                    Cerevisiae_cse4  
gal_cse4_353_r2                                    Cerevisiae_cse4  
rad53 _r1                                         Cerevisiae_rad53  
rad53_r2                                          Cerevisiae_rad53  
cdc16cdc28whi5stb1_rep1              Cerevisiae_cdc16cdc28whi5stb1  
cdc16cdc28whi5stb1_rep2              Cerevisiae_cdc16cdc28whi5stb1  
cdc28_whi5_stb1_r1                  Cerevisiae_cdc28whi5stb1_elutr  
cdc28_whi5_stb1_r1                        Cerevisiae_cdc28whi5stb1  
cdc28_whi5_stb1_r2                  Cerevisiae_cdc28whi5stb1_elutr  
cdc28_whi5_stb1_r2                        Cerevisiae_cdc28whi5stb1  
gal_sic1_3P_r1                              GAL-SIC1Δ3P_biological  
gal_sic1_3P_r2                              GAL-SIC1Δ3P_biological  
gal_sic1_3P_yox1_yhp1_nrm1  GAL-SIC1Δ3P_nrm1Δyhp1Δyox1Δ_biological  
mt_r1                                      Cerevisiae_CyclinMutant  
mt_r2                                      Cerevisiae_CyclinMutant  
wt_r1                                          Cerevisiae_WildType  
wt_r2                                          Cerevisiae_WildType  
cdc14                                                      cdc14-3  
cdc15                                                      cdc15-2

In [23]:
def get_geo_data(geo_accession, outdir, log_file):

    gse_dir = os.path.join(outdir, geo_accession)
    
    gse = GEOparse.get_GEO(geo=geo_accession, destdir=gse_dir, silent=True)
    
    gsms_to_download = []
    gsms_to_ignore = []
    
    series_metadata = pd.DataFrame.from_dict([gse.metadata])
    series_metadata_cols = ['!Series_{}'.format(col_name) for col_name in series_metadata.columns]
    series_metadata.columns = series_metadata_cols
    series_metadata = series_metadata.T
    
    if '!Series_relation' in series_metadata.index.tolist():
        for item in series_metadata.loc['!Series_relation'][0]:
            if 'Reanalysis' in item:
                #ex: Reanalysis of: GSM218597
                ignored_gsm = item.split(":")[1].replace(' ', '')
                gsms_to_ignore.append(ignored_gsm)
        for item in series_metadata.loc['!Series_sample_id'][0]:
            gsms_to_download.append(item)
    
    sample_metadata_list = []
    for gsm_name, gsm in gse.gsms.items():
        if 'relation' in gsm.metadata.keys():
            single_sample_metadata = []
            for item in gsm.metadata['relation']:
                if 'Reanalysis' in item:
                    gsms_to_ignore.append(gsm.metadata['geo_accession'][0])
                else:
                    gsms_to_download.append(gsm.metadata['geo_accession'][0])
                    single_sample_metadata.append(gsm.metadata)
            if len(single_sample_metadata) > 0:
                sample_metadata_list.append(single_sample_metadata[0])
        else:
            gsms_to_download.append(gsm.metadata['geo_accession'][0])
            sample_metadata_list.append(gsm.metadata)

    sample_metadata = pd.DataFrame.from_dict(sample_metadata_list)
    sample_cols = ['!Sample_{}'.format(col_name) for col_name in sample_metadata.columns]
    sample_metadata.columns = sample_cols
    sample_metadata = sample_metadata.T
    sample_metadata = sample_metadata.fillna('')
    sample_metadata = sample_metadata.apply(list_to_string, axis=0)
    sample_metadata = sample_metadata.reset_index()

#     take only unique gsm_to_download
    unqiue_ignore_list = list(set(gsms_to_ignore))
    gsms_to_download = list(set(gsms_to_download))
    unique_gsm_downloads = [n for n in gsms_to_download if n not in unqiue_ignore_list]

#     make !Series_sample_id equal to gsm download list
    series_metadata.at['!Series_sample_id',0] = unique_gsm_downloads
    series_metadata = series_metadata.apply(list_to_string, axis=0)
    series_metadata = series_metadata.reset_index()
    
    series_matrix_data = gse.pivot_samples('VALUE')
    series_matrix_data = series_matrix_data.loc[:, unique_gsm_downloads]
    series_matrix_data = series_matrix_data.reset_index().T.reset_index().T

    for gsm in unique_gsm_downloads:
        gse.gsms[gsm].download_supplementary_files(directory=os.path.join(gse_dir, 'raw'))

        # move file and delete dir and contents
        for supfile in gse.gsms[gsm].metadata['supplementary_file']:
            if 'CEL' in supfile:
                file_name = os.path.basename(supfile)
        file_name = file_name.replace(',', '_')
        file_name = file_name.replace(' ', '_')
        dir_name = 'Supp_{}_{}'.format(gse.gsms[gsm].metadata['geo_accession'][0], gse.gsms[gsm].metadata['title'][0])
        dir_name = dir_name.replace(',', '_')
        dir_name = dir_name.replace(' ', '_')
        os.rename(os.path.join(gse_dir, 'raw', dir_name, file_name), os.path.join(gse_dir, 'CEL_RAW', file_name))
        shutil.rmtree(os.path.join(gse_dir, 'raw', dir_name))

    with open(log_file, 'a') as f:
        gsm_meta = len(unique_gsm_downloads)
        gsm_down = len(os.listdir(os.path.join(gse_dir, 'raw')))
        meta_down_diff = abs(gsm_meta - gsm_down)
        f.write('{}\n\tNumber of GSMs in metadata: {}\n\tNumber of GSMs downloaded: {}\n\t\tdifference: {}\n'.format(geo_accession, gsm_meta, 
                                                                                                       gsm_down, meta_down_diff))
        
    data_dict = [series_metadata, sample_metadata, series_matrix_data]

    with open(os.path.join(gse_dir, geo_accession + '_series_matrix.txt'), 'a') as f:
        for idx, data in enumerate(data_dict):
            if idx == 0:
                data.to_csv(f, header=False, index=False, sep='\t')
                f.write('\n')
            if idx == 1:
                x = 2
                data.to_csv(f, header=False, index=False, sep='\t')
            if idx == 2:
                f.write('!series_matrix_table_begin\n')
                data.to_csv(f, header=False, index=False, sep='\t')
                f.write('!series_matrix_table_end\n')

In [24]:
dt_string = datetime.now().strftime("%Y%m%d%H%M%S")
log_name = '{}_file_count.log'.format(dt_string)
log_path = os.path.join(data_dir, log_name)
with open(log_path, 'a') as f:
    f.write(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    f.write('\n')

for gse_id in set(geo_datasets['GEO Accession'].tolist()):
    print(gse_id)
    get_geo_data(gse_id, data_dir, log_path)

GSE8799


100%|██████████| 1.60M/1.60M [00:00<00:00, 3.88MB/s]
100%|██████████| 57.5k/57.5k [00:00<00:00, 394kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.23MB/s]
100%|██████████| 58.1k/58.1k [00:00<00:00, 356kB/s] 
100%|██████████| 1.55M/1.55M [00:00<00:00, 4.07MB/s]
100%|██████████| 57.9k/57.9k [00:00<00:00, 422kB/s]
100%|██████████| 1.59M/1.59M [00:00<00:00, 4.00MB/s]
100%|██████████| 57.7k/57.7k [00:00<00:00, 452kB/s]
100%|██████████| 1.56M/1.56M [00:00<00:00, 4.07MB/s]
100%|██████████| 57.7k/57.7k [00:00<00:00, 421kB/s]
100%|██████████| 1.57M/1.57M [00:00<00:00, 4.35MB/s]
100%|██████████| 57.9k/57.9k [00:00<00:00, 226kB/s] 
100%|██████████| 1.57M/1.57M [00:00<00:00, 3.85MB/s]
100%|██████████| 57.7k/57.7k [00:00<00:00, 387kB/s]
100%|██████████| 1.58M/1.58M [00:00<00:00, 3.97MB/s]
100%|██████████| 57.7k/57.7k [00:00<00:00, 408kB/s]
100%|██████████| 1.58M/1.58M [00:00<00:00, 4.13MB/s]
100%|██████████| 57.9k/57.9k [00:00<00:00, 462kB/s]
100%|██████████| 1.58M/1.58M [00:00<00:00, 3.85MB/s]


GSE32974


100%|██████████| 1.05M/1.05M [00:00<00:00, 3.19MB/s]
100%|██████████| 1.01M/1.01M [00:00<00:00, 3.40MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.45MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.64MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 3.36MB/s]
100%|██████████| 1.04M/1.04M [00:00<00:00, 3.57MB/s]
100%|██████████| 959k/959k [00:00<00:00, 3.23MB/s] 
100%|██████████| 1.06M/1.06M [00:00<00:00, 3.49MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.02MB/s]
100%|██████████| 1.03M/1.03M [00:00<00:00, 3.20MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.55MB/s]
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.02MB/s]
100%|██████████| 984k/984k [00:00<00:00, 3.35MB/s] 
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.53MB/s]
100%|██████████| 982k/982k [00:00<00:00, 3.01MB/s] 
100%|██████████| 1.01M/1.01M [00:00<00:00, 3.17MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.48MB/s]
100%|██████████| 1.06M/1.06M [00:00<00:00, 3.71MB/s]
100%|██████████| 971k/971k [00:00<00:00, 3.30MB/s

GSE96997


100%|██████████| 1.06M/1.06M [00:00<00:00, 3.57MB/s]
100%|██████████| 1.04M/1.04M [00:00<00:00, 3.78MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.31MB/s]
100%|██████████| 898k/898k [00:00<00:00, 3.47MB/s] 
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.75MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.45MB/s]
100%|██████████| 1.03M/1.03M [00:00<00:00, 3.56MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.56MB/s]
100%|██████████| 1.06M/1.06M [00:00<00:00, 3.37MB/s]
100%|██████████| 1.06M/1.06M [00:00<00:00, 3.67MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 3.67MB/s]
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.43MB/s]
100%|██████████| 953k/953k [00:00<00:00, 3.73MB/s]
100%|██████████| 950k/950k [00:00<00:00, 3.56MB/s] 
100%|██████████| 922k/922k [00:00<00:00, 3.52MB/s] 
100%|██████████| 1.04M/1.04M [00:00<00:00, 3.91MB/s]
100%|██████████| 1.04M/1.04M [00:01<00:00, 561kB/s] 
100%|██████████| 1.05M/1.05M [00:01<00:00, 665kB/s]
100%|██████████| 950k/950k [00:02<00:00, 421kB/s]  


GSE75694


100%|██████████| 976k/976k [00:00<00:00, 2.91MB/s] 
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.03MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.32MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.52MB/s]
100%|██████████| 1.04M/1.04M [00:00<00:00, 3.21MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 3.08MB/s]
100%|██████████| 1.03M/1.03M [00:00<00:00, 3.51MB/s]
100%|██████████| 1.01M/1.01M [00:00<00:00, 3.37MB/s]
100%|██████████| 1.01M/1.01M [00:00<00:00, 3.36MB/s]
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.23MB/s]
100%|██████████| 1.04M/1.04M [00:00<00:00, 3.39MB/s]
100%|██████████| 1.01M/1.01M [00:00<00:00, 2.74MB/s]
100%|██████████| 1.06M/1.06M [00:00<00:00, 3.35MB/s]
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.33MB/s]
100%|██████████| 1.00M/1.00M [00:00<00:00, 2.99MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.54MB/s]
100%|██████████| 1.02M/1.02M [00:00<00:00, 3.41MB/s]
100%|██████████| 1.05M/1.05M [00:00<00:00, 2.91MB/s]
100%|██████████| 989k/989k [00:00<00:00, 3.28MB

GSE49650


100%|██████████| 982k/982k [00:00<00:00, 3.02MB/s] 
100%|██████████| 904k/904k [00:00<00:00, 2.77MB/s] 
100%|██████████| 999k/999k [00:00<00:00, 3.34MB/s] 
100%|██████████| 947k/947k [00:00<00:00, 3.40MB/s] 
100%|██████████| 981k/981k [00:00<00:00, 3.39MB/s] 
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.21MB/s]
100%|██████████| 950k/950k [00:00<00:00, 3.08MB/s] 
100%|██████████| 972k/972k [00:00<00:00, 3.45MB/s] 
100%|██████████| 960k/960k [00:00<00:00, 3.25MB/s] 
100%|██████████| 965k/965k [00:00<00:00, 3.40MB/s] 
100%|██████████| 1.03M/1.03M [00:00<00:00, 3.14MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 3.22MB/s]
100%|██████████| 996k/996k [00:00<00:00, 2.86MB/s] 
100%|██████████| 972k/972k [00:00<00:00, 2.64MB/s] 
100%|██████████| 994k/994k [00:00<00:00, 3.35MB/s] 
100%|██████████| 1.00M/1.00M [00:00<00:00, 3.41MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 3.42MB/s]
100%|██████████| 931k/931k [00:00<00:00, 3.08MB/s] 
100%|██████████| 0.98M/0.98M [00:00<00:00, 3.46MB/s]
100%|█

In [13]:
# data_dir = '/Users/robertmoseley/Desktop/Haase_lab/mass_norm/data/GEO_data/test'

# dt_string = datetime.now().strftime("%Y%m%d%H%M%S")
# log_name = '{}_file_count.log'.format(dt_string)
# log_path = os.path.join(data_dir, log_name)
# with open(log_path, 'a') as f:
#     f.write(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
#     f.write('\n')

# for gse_id in ['GSE75694', 'GSE8799', 'GSE96997']:
#     get_geo_data(gse_id, data_dir, log_path)

In [25]:
# data_dir = '/Users/robertmoseley/Desktop/Haase_lab/mass_norm/data/GEO_data/test'
# dt_string = datetime.now().strftime("%Y%m%d%H%M%S")
# log_name = '{}_file_count.log'.format(dt_string)
# log_path = os.path.join(data_dir, log_name)
# with open(log_path, 'a') as f:
#     f.write(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
#     f.write('\n')

# get_geo_data('GSE96997', data_dir, log_path)